# Introduction
In this notebook we present our code for our capstone project in an easy to read and understandable way.

* ___Say something about the project___

* ___Say something about the notebook structure___

* ___Any additional notes?___

# Setup

* ___Optional: Add subtext to every header___

## Basic setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2 # needed to get database exception errors when uploading dataframe
import sql_functions as sf # own collection of functions to manage connections to our SQL database

In [2]:
# Define variables
schema = 'capstone_kueblbeck'
engine = sf.get_engine()

In [3]:
# Other settings
pd.options.display.max_columns = 40
pd.options.display.float_format = "{:,.2f}".format

## Prepare data and export to SQL

### Import data from external files

In [4]:
# Import "Lagerbestand"
lagerbestand = pd.read_csv('./data/Lagerbestand_02.Juni.2023.txt', decimal=',')
lagerbestand = lagerbestand.drop(lagerbestand.index[0])

/var/folders/kv/8g80w4s52_ldz0hf8y_2jgw40000gn/T/ipykernel_20669/2934447077.py:2: DtypeWarning: Columns (0,2,5,10,11,13,15,17,19,21,23,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  lagerbestand = pd.read_csv('./data/Lagerbestand_02.Juni.2023.txt', decimal=',')


In [5]:
# Import "Lieferanten"
lieferanten = pd.read_excel('./data/Lieferantenübersicht.xlsx')
lieferanten = lieferanten.drop(lieferanten.index[0])

In [6]:
# Import "Verkäufe"
verkaeufe = pd.read_csv('./data/Abverkauf_DE_2022_8410.csv', decimal=',')
verkaeufe = verkaeufe.drop(verkaeufe.index[0])

/var/folders/kv/8g80w4s52_ldz0hf8y_2jgw40000gn/T/ipykernel_20669/3738684025.py:2: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  verkaeufe = pd.read_csv('./data/Abverkauf_DE_2022_8410.csv', decimal=',')


### Correct datatypes for each table (if necessary)

#### Lagerbestand

In [7]:
# Change date columns from string to date type
date_columns = ["Ltz. VK ges.", "Ltz. VK WEN", "Ltz. VK RGB", "Ltz. VK AMB", "Ltz. VK CHA", "Ltz. VK STR", "Ltz. VK PAS", "Ltz. VK LAN", "Ltz. VK MÜH", "Ltz. VK ROS"]

for column in date_columns:
        lagerbestand[column] = pd.to_datetime(lagerbestand[column], format='%d.%m.%Y', errors='coerce')

In [8]:
# Change selected number columns from string to float type
numeric_columns = ['Gesamt', 'WEN', 'RGB', 'AMB', 'CHA', 'STR', 'PAS', 'LAN', 'MÜH', 'ROS']

for column in numeric_columns:
    lagerbestand[column] = pd.to_numeric(lagerbestand[column].str.replace('.', '').str.replace(',','.'), errors='coerce')

/var/folders/kv/8g80w4s52_ldz0hf8y_2jgw40000gn/T/ipykernel_20669/2712846017.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lagerbestand[column] = pd.to_numeric(lagerbestand[column].str.replace('.', '').str.replace(',','.'), errors='coerce')


In [9]:
# Special treatment for columns 'Basispreis' and 'Basispr. Summe'
numeric_columns_basis = ['Basispreis', 'Basispr. Summe']

for column in numeric_columns_basis:
    lagerbestand[column] = pd.to_numeric(lagerbestand[column].str.replace('.', '').str.replace(',','.'))

/var/folders/kv/8g80w4s52_ldz0hf8y_2jgw40000gn/T/ipykernel_20669/727319556.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lagerbestand[column] = pd.to_numeric(lagerbestand[column].str.replace('.', '').str.replace(',','.'))


In [10]:
# Change 'Lfnr' and 'VPE' to Integer type, as these categorical numbers
lagerbestand['Lfnr'] = lagerbestand['Lfnr'].astype(int)
lagerbestand['VPE'] = lagerbestand['VPE'].astype(int)

#### Lieferanten

In [11]:
# Sync datatype in column 'Lfnr' to table 'Lagerbestand'
lieferanten['Lfnr'] = lieferanten['Lfnr'].astype(int)

#### Verkaeufe

In [12]:
# Change selected number columns from string to float type
numeric_columns = ["Gesamt", "WEN", "RGB", "AMB", "CHA", "STR", "PAS", "LAN", "MÜH", "ROS"]

for column in numeric_columns:
    verkaeufe[column] = pd.to_numeric(verkaeufe[column].str.replace('.', ''), errors='coerce')

/var/folders/kv/8g80w4s52_ldz0hf8y_2jgw40000gn/T/ipykernel_20669/1413186216.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  verkaeufe[column] = pd.to_numeric(verkaeufe[column].str.replace('.', ''), errors='coerce')


In [13]:
# Sync datatype in column 'Lfr.' to table 'Lagerbestand'
verkaeufe['Lfr.'] = verkaeufe['Lfr.'].astype(int)

#### Check Dataframes before Export

In [14]:
display(lagerbestand.head(2))
display(lagerbestand.info())

,Lfnr,Artnr,Index,Beschr.,BKZ,VPE,St.gr.,Ltz. VK ges.,Basispreis,Basispr. Summe,Gesamt,WEN,Ltz. VK WEN,RGB,Ltz. VK RGB,AMB,Ltz. VK AMB,CHA,Ltz. VK CHA,STR,Ltz. VK STR,PAS,Ltz. VK PAS,LAN,Ltz. VK LAN,MÜH,Ltz. VK MÜH,ROS,Ltz. VK ROS
1,430,08.607.83,000,"N CORSA D 1,0L 44KW BJ 2006",8,1,L20,2018-11-07,75.36,0.00,0.00,0.00,NaT,0.00,2018-11-07,0.00,2017-08-08,0.00,2016-04-29,0.00,2017-12-14,0.00,NaT,0.00,2017-08-31,0.00,NaT,0.00,2014-03-26
2,430,08.607.81,000,"M CORSA D 1,4L 66KW BJ 2006",8,1,L20,2021-08-04,93.63,0.00,0.00,0.00,2021-08-04,0.00,2017-07-10,0.00,2021-04-26,0.00,2021-04-28,0.00,2018-09-13,0.00,2019-07-31,0.00,2016-06-24,0.00,NaT,0.00,2015-05-04


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357829 entries, 1 to 357829
Data columns (total 29 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Lfnr            357829 non-null  int64         
 1   Artnr           357829 non-null  object        
 2   Index           357829 non-null  object        
 3   Beschr.         357829 non-null  object        
 4   BKZ             357829 non-null  object        
 5   VPE             357829 non-null  int64         
 6   St.gr.          357829 non-null  object        
 7   Ltz. VK ges.    338128 non-null  datetime64[ns]
 8   Basispreis      357829 non-null  float64       
 9   Basispr. Summe  357829 non-null  float64       
 10  Gesamt          325061 non-null  float64       
 11  WEN             325061 non-null  float64       
 12  Ltz. VK WEN     204992 non-null  datetime64[ns]
 13  RGB             98303 non-null   float64       
 14  Ltz. VK RGB     124822 non-null  dat

None

In [15]:
display(lieferanten.head(2))
display(lieferanten.info())

,Lfnr,Beschreibung
1,0,BOSCH
2,1,HELLA


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 1 to 378
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Lfnr          378 non-null    int64 
 1   Beschreibung  376 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.0+ KB


None

In [16]:
display(verkaeufe.head(2))
display(verkaeufe.info())

,Lfr.,Art.nr.,Ind.,Beschreibung,Gesamt,WEN,RGB,STR,PAS,AMB,CHA,LAN,MÜH,ROS
1,0,1928498680,0.00,BUCHSENKONTAKT,"38,400.00","38,400.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,52,50266496,0.00,"KABELBAND 300X4,8MM SCHWARZ","28,200.00","25,000.00",0.00,0.00,0.00,0.00,"1,400.00",0.00,"1,500.00",300.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66009 entries, 1 to 66009
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Lfr.          66009 non-null  int64  
 1   Art.nr.       66009 non-null  object 
 2   Ind.          66009 non-null  float64
 3   Beschreibung  66009 non-null  object 
 4   Gesamt        65535 non-null  float64
 5   WEN           65535 non-null  float64
 6   RGB           65535 non-null  float64
 7   STR           65535 non-null  float64
 8   PAS           65535 non-null  float64
 9   AMB           65535 non-null  float64
 10  CHA           65535 non-null  float64
 11  LAN           65535 non-null  float64
 12  MÜH           65535 non-null  float64
 13  ROS           65535 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 7.1+ MB


None

### Export to SQL

Note: The following code blocks will be fully commented out as the dataframes are already uploaded. Running these codes would therefore cause an error due to restrictions on our SQL server.

In [17]:
# # Import Lagerbestand data into SQL 
# table_name = 'lagerbestand'

# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         lagerbestand.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schema that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [18]:
# # Import 'Lieferanten' data into SQL 
# table_name = 'lieferanten'

# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         lieferanten.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schema that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [19]:
# # Import 'Verkaeufe' data into SQL 
# table_name = 'verkäufe'

# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         verkaeufe.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schema that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

# Data Cleaning

## Loading dataframes

In [20]:
# Lagerbestand
sql_query = f'select * from {schema}.lagerbestand'
df_lagerbestand = sf.get_dataframe(sql_query)

In [21]:
# Lieferanten
sql_query = f'select * from {schema}.lieferanten'
df_lieferanten = sf.get_dataframe(sql_query)

In [22]:
# Verkaeufe
sql_query = f'select * from {schema}.verkäufe'
df_verkaeufe = sf.get_dataframe(sql_query)

## df_lagerbestand

In [23]:
# Adjust column names
df_lagerbestand.columns = df_lagerbestand.columns.str.lower()
df_lagerbestand.columns = [col.replace(" ", "_") for col in df_lagerbestand.columns.tolist()]
df_lagerbestand.columns = [col.replace(".", "") for col in df_lagerbestand.columns.tolist()]

In [24]:
# Change names of selected columns for clearer identification
new_columns = {'beschr':'beschreibung',
               'bkz':'bestellkennzeichen',
               'vpe':'verp_einheit',
               'stgr':'stat_gruppe',
               'gesamt':'gesamt_lager',
               'wen':'wen_lager',
               'rgb':'rgb_lager',
               'str':'str_lager',
               'pas':'pas_lager',
               'amb':'amb_lager',
               'cha':'cha_lager',
               'lan':'lan_lager',
               'müh':'müh_lager',
               'ros':'ros_lager'}

df_lagerbestand = df_lagerbestand.rename(columns=new_columns)

In [25]:
# Additional changing of datatypes for index column (Integer due to it being categorical)
df_lagerbestand['index'] = df_lagerbestand['index'].astype(int)

## df_lieferanten

In [26]:
# Adjust column names
df_lieferanten.columns = df_lieferanten.columns.str.lower()
df_lieferanten.columns = [col.replace(" ", "_") for col in df_lieferanten.columns.tolist()]
df_lieferanten.columns = [col.replace(".", "") for col in df_lieferanten.columns.tolist()]

In [27]:
# Change names of selected columns for clearer identification
df_lieferanten = df_lieferanten.rename(columns={'beschreibung':'lieferant'})

## df_verkaeufe

In [28]:
# Adjust column names
df_verkaeufe.columns = df_verkaeufe.columns.str.lower()
df_verkaeufe.columns = [col.replace(" ", "_") for col in df_verkaeufe.columns.tolist()]
df_verkaeufe.columns = [col.replace(".", "") for col in df_verkaeufe.columns.tolist()]

In [29]:
# Change names of selected columns for clearer identification
new_columns = {'lfr':'lfnr',
               'ind': 'index',
               'gesamt':'gesamt_vk',
               'wen':'wen_vk',
               'rgb':'rgb_vk',
               'str':'str_vk',
               'pas':'pas_vk',
               'amb':'amb_vk',
               'cha':'cha_vk',
               'lan':'lan_vk',
               'müh':'müh_vk',
               'ros':'ros_vk'}

df_verkaeufe = df_verkaeufe.rename(columns=new_columns)

In [30]:
# Additional changing of datatypes for index column (Integer due to it being categorical)
df_verkaeufe['index'] = df_verkaeufe['index'].astype(int)

## Check dataframes

In [37]:
display(df_lagerbestand.head(2))
display(df_lagerbestand.info())

,lfnr,artnr,index,beschreibung,bestellkennzeichen,verp_einheit,stat_gruppe,ltz_vk_ges,basispreis,basispr_summe,gesamt_lager,wen_lager,ltz_vk_wen,rgb_lager,ltz_vk_rgb,amb_lager,ltz_vk_amb,cha_lager,ltz_vk_cha,str_lager,ltz_vk_str,pas_lager,ltz_vk_pas,lan_lager,ltz_vk_lan,müh_lager,ltz_vk_müh,ros_lager,ltz_vk_ros
0,430,08.607.83,0,"N CORSA D 1,0L 44KW BJ 2006",8,1,L20,2018-11-07,75.36,0.00,0.00,0.00,NaT,0.00,2018-11-07,0.00,2017-08-08,0.00,2016-04-29,0.00,2017-12-14,0.00,NaT,0.00,2017-08-31,0.00,NaT,0.00,2014-03-26
1,430,08.607.81,0,"M CORSA D 1,4L 66KW BJ 2006",8,1,L20,2021-08-04,93.63,0.00,0.00,0.00,2021-08-04,0.00,2017-07-10,0.00,2021-04-26,0.00,2021-04-28,0.00,2018-09-13,0.00,2019-07-31,0.00,2016-06-24,0.00,NaT,0.00,2015-05-04


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357829 entries, 0 to 357828
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   lfnr                357829 non-null  int64         
 1   artnr               357829 non-null  object        
 2   index               357829 non-null  int64         
 3   beschreibung        357829 non-null  object        
 4   bestellkennzeichen  357829 non-null  object        
 5   verp_einheit        357829 non-null  int64         
 6   stat_gruppe         357829 non-null  object        
 7   ltz_vk_ges          338128 non-null  datetime64[ns]
 8   basispreis          357829 non-null  float64       
 9   basispr_summe       357829 non-null  float64       
 10  gesamt_lager        325061 non-null  float64       
 11  wen_lager           325061 non-null  float64       
 12  ltz_vk_wen          204992 non-null  datetime64[ns]
 13  rgb_lager           98303 non

None

In [38]:
display(df_lieferanten.head(2))
display(df_lieferanten.info())

,lfnr,lieferant
0,0,BOSCH
1,1,HELLA


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lfnr       378 non-null    int64 
 1   lieferant  376 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.0+ KB


None

In [39]:
display(df_verkaeufe.head(2))
display(df_verkaeufe.info())

,lfnr,artnr,index,beschreibung,gesamt_vk,wen_vk,rgb_vk,str_vk,pas_vk,amb_vk,cha_vk,lan_vk,müh_vk,ros_vk
0,0,1928498680,0,BUCHSENKONTAKT,"38,400.00","38,400.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,52,50266496,0,"KABELBAND 300X4,8MM SCHWARZ","28,200.00","25,000.00",0.00,0.00,0.00,0.00,"1,400.00",0.00,"1,500.00",300.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66009 entries, 0 to 66008
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lfnr          66009 non-null  int64  
 1   artnr         66009 non-null  object 
 2   index         66009 non-null  int64  
 3   beschreibung  66009 non-null  object 
 4   gesamt_vk     65535 non-null  float64
 5   wen_vk        65535 non-null  float64
 6   rgb_vk        65535 non-null  float64
 7   str_vk        65535 non-null  float64
 8   pas_vk        65535 non-null  float64
 9   amb_vk        65535 non-null  float64
 10  cha_vk        65535 non-null  float64
 11  lan_vk        65535 non-null  float64
 12  müh_vk        65535 non-null  float64
 13  ros_vk        65535 non-null  float64
dtypes: float64(10), int64(2), object(2)
memory usage: 7.1+ MB


None

# Merging tables to df_master

In [34]:
# Merging df_lagerbestand and df_lieferanten
df_master = df_lagerbestand.merge(df_lieferanten, how='left', on='lfnr')

In [35]:
# Merging df_verkaeufe on df_master
df_master = df_master.merge(df_verkaeufe, how='left', on=['lfnr', 'artnr', 'index', 'beschreibung'])

In [36]:
# Adjusting column positions
new_column_order = ['lfnr','lieferant', 'artnr', 'beschreibung', 'index', 'bestellkennzeichen',
       'verp_einheit', 'stat_gruppe', 'ltz_vk_ges', 'basispreis',
       'basispr_summe', 'gesamt_lager', 'wen_lager', 'ltz_vk_wen', 'rgb_lager',
       'ltz_vk_rgb', 'amb_lager', 'ltz_vk_amb', 'cha_lager', 'ltz_vk_cha',
       'str_lager', 'ltz_vk_str', 'pas_lager', 'ltz_vk_pas', 'lan_lager',
       'ltz_vk_lan', 'müh_lager', 'ltz_vk_müh', 'ros_lager', 'ltz_vk_ros',
        'gesamt_vk', 'wen_vk', 'rgb_vk', 'str_vk', 'pas_vk',
       'amb_vk', 'cha_vk', 'lan_vk', 'müh_vk', 'ros_vk']

df_master = df_master.reindex(columns = new_column_order)

## Check df_master

In [40]:
display(df_master.head(2))
display(df_master.info())

,lfnr,lieferant,artnr,beschreibung,index,bestellkennzeichen,verp_einheit,stat_gruppe,ltz_vk_ges,basispreis,basispr_summe,gesamt_lager,wen_lager,ltz_vk_wen,rgb_lager,ltz_vk_rgb,amb_lager,ltz_vk_amb,cha_lager,ltz_vk_cha,str_lager,ltz_vk_str,pas_lager,ltz_vk_pas,lan_lager,ltz_vk_lan,müh_lager,ltz_vk_müh,ros_lager,ltz_vk_ros,gesamt_vk,wen_vk,rgb_vk,str_vk,pas_vk,amb_vk,cha_vk,lan_vk,müh_vk,ros_vk
0,430,Eberspächer Abgas,08.607.83,"N CORSA D 1,0L 44KW BJ 2006",0,8,1,L20,2018-11-07,75.36,0.00,0.00,0.00,NaT,0.00,2018-11-07,0.00,2017-08-08,0.00,2016-04-29,0.00,2017-12-14,0.00,NaT,0.00,2017-08-31,0.00,NaT,0.00,2014-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,430,Eberspächer Abgas,08.607.81,"M CORSA D 1,4L 66KW BJ 2006",0,8,1,L20,2021-08-04,93.63,0.00,0.00,0.00,2021-08-04,0.00,2017-07-10,0.00,2021-04-26,0.00,2021-04-28,0.00,2018-09-13,0.00,2019-07-31,0.00,2016-06-24,0.00,NaT,0.00,2015-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 357829 entries, 0 to 357828
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   lfnr                357829 non-null  int64         
 1   lieferant           357828 non-null  object        
 2   artnr               357829 non-null  object        
 3   beschreibung        357829 non-null  object        
 4   index               357829 non-null  int64         
 5   bestellkennzeichen  357829 non-null  object        
 6   verp_einheit        357829 non-null  int64         
 7   stat_gruppe         357829 non-null  object        
 8   ltz_vk_ges          338128 non-null  datetime64[ns]
 9   basispreis          357829 non-null  float64       
 10  basispr_summe       357829 non-null  float64       
 11  gesamt_lager        325061 non-null  float64       
 12  wen_lager           325061 non-null  float64       
 13  ltz_vk_wen          204992 no

None